# Lab 5: Logistic Classification

## Reminder: Logistic Regression
### Hypothesis
$$ H(X) = \cfrac{1}{1+e^{-W^TX}} $$

### Cost
$$ cost(W) = -\cfrac{1}{m} \sum y\log(H(x)) + (1-y)(\log(1-H(x))) $$
- If $ y \simeq H(x) $, cost is near 0.
- if $ y \neq   H(x) $, cost is high.

### Weight Update via Gradient Descent
$$ W := W - \alpha \cfrac{\partial}{\partial W} cost(W) $$
- $\alpha$: Learning rate

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Training Data

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

다음의 분류 문제 상황을 가정하자.  
```각 학생이 강의를 들은 시간과 코드 랩에서 작업한 시간을 입력으로 받는다. 학생이 강의에서 Pass를 받을지 Fail을 받을지 예측해라. 예를 들어, 첫 번째(index 0)학생은 강의를 1시간 보았고 2시간을 코딩에 썼다. 그리고 Fail을 받았다(\[0\]).```

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

항상 그렇듯, 데이서를 `torch.Tensor` 형태로 바꾸어야 한다.

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis
$$ H(X) = \cfrac{1}{1 + e^{-W^TX}} $$

PyTorch는 지수함수와 닮은 `torch.exp()`함수를 가지고 있다.

In [6]:
print('e^1 equals:', torch.exp(torch.FloatTensor([1])))

e^1 equals: tensor([2.7183])


위의 함수를 이용하여 가설(hypothesis) 함수를 간편하게 계산할 수 있다.

In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


또는 시그모이드 함수와 비슷한 `torch.sigmoid()`를 사용할 수 있다.

In [9]:
print('1/(1 + e^(-1)) equals:', torch.sigmoid(torch.FloatTensor([1])))

1/(1 + e^(-1)) equals: tensor([0.7311])


이제 가설(hypothesis) 함수의 코드를 더 단순하게 쓸 수 있다.

In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Funtion (Low-level)
$$ cost(W) = - \cfrac{1}{m} \sum y\log(H(x)) + (1-y)(\log(1-H(x))) $$

In [11]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


하나의 원소에 대해서 손실(loss)은 다음과 같이 계산될 수 있다:

In [12]:
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

배치(batch)의 전체 손실(loss)를 구하기 위해서는 단순히 전체 벡터를 넣으면 된다.

In [13]:
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


이후, `.mean()`을 이용하여 손실의 평균을 구한다.

In [14]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


## Computing the Cost Function with `F.binary_cross_entropy`
이진 분류는 현실에서 매우 자주 쓰이기 때문에, PyTorch는 `F.binary_cross_entropy`라는 간단한 함수를 제공한다.

In [15]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Traing with Low-level Binary Cross Entropy Loss

In [16]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [17]:
# 모델 초기화
W = torch.zeros((2,1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr= 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #Cost 계산 (Low-level implementation)
  hypothesis = torch.sigmoid(x_train.matmul(W) + b) # 또는 .mm()이나 @기호를 사용할 수 있음
  cost = -(y_train * torch.log(hypothesis) +
           (1-y_train) * torch.log(1 - hypothesis)).mean()

  #Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))


Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Training with `F.binary_cross_entropy`

In [18]:
# 모델 초기화
W = torch.zeros((2,1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr= 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b) # 또는 .mm()이나 @기호를 사용할 수 있음
  cost = F.binary_cross_entropy(hypothesis, y_train)

  #Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Loading Real Data

In [19]:
import numpy as np

In [20]:
xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [21]:
print(x_train[0:5])
print(y_train[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


## Training with Real Data using `F.binary_cross_entropy`

In [22]:
#모델 초기화
W = torch.zeros((8, 1), requires_grad= True)
b = torch.zeros(1 , requires_grad= True)

#optimizer 설정
optimizer = optim.SGD([W,b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
  #Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  #H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #10번 마다 로그 출력
  if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488209
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


## Checking the Accuracy our Model - 모델의 정확도 확인하기
모델이 훈련 데이터에 얼마나 최적화되었는지 확인해보자.

In [23]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.4103],
        [0.9242],
        [0.2300],
        [0.9411],
        [0.1772]], grad_fn=<SliceBackward0>)


**hypothesis**(0부터 1사이의 실수)를 **binary prediction**(0또는 1)로 바꿀 수 있다.

In [24]:
prediction = hypothesis >= torch.FloatTensor([0.5]) #hypothesis가 0.5보다 큰지(bool)
print(prediction[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])


이제 이것을 `y_train`과 비교해 볼 수 있다.

In [25]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [26]:
correct_prediction = prediction.float() == y_train #prediction의 각 요소가 y_train과 같은지
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


마지막으로, 일치하는 예측의 개수를 세고 전체 예측의 개수로 나누어 정확도(accuracy)를 구할 수 있다.

In [27]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 76.68% for the training set.


## Optional: High-level Implementation with `nn.Module`

In [28]:
class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(8, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, x):
    return self.sigmoid(self.linear(x))

model = BinaryClassifier()

In [30]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr= 1)

nb_epoches = 100
for epoch in range(nb_epochs + 1):
  #H(x) 계산
  hypothesis = model(x_train)
  #cost 계산
  cost = F.binary_cross_entropy(hypothesis, y_train)

  #H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = (prediction.float() == y_train)
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.704829 Accuracy 45.72%
Epoch   10/100 Cost: 0.572392 Accuracy 67.59%
Epoch   20/100 Cost: 0.539563 Accuracy 73.25%
Epoch   30/100 Cost: 0.520041 Accuracy 75.89%
Epoch   40/100 Cost: 0.507561 Accuracy 76.15%
Epoch   50/100 Cost: 0.499125 Accuracy 76.42%
Epoch   60/100 Cost: 0.493177 Accuracy 77.21%
Epoch   70/100 Cost: 0.488846 Accuracy 76.81%
Epoch   80/100 Cost: 0.485612 Accuracy 76.28%
Epoch   90/100 Cost: 0.483146 Accuracy 76.55%
Epoch  100/100 Cost: 0.481234 Accuracy 76.81%
